# Notebook 04: Molecular Collision Dynamics

## 1. Introduction: The Microscopic View

Chemical kinetics averages over billions of molecules. **Molecular reaction dynamics** looks at individual collision events. It asks: what happens when two specific molecules collide with a specific energy and orientation?

### Key Concepts
1.  **Molecular Beams**: Experimental technique to study single collisions without interference from bulk gas.
2.  **Potential Energy Surfaces (PES)**: The "landscape" of energy that dictates how atoms move.
3.  **Trajectories**: The path a reaction takes on the PES.

### Learning Objectives
1.  Describe how molecular beam experiments measure differential cross-sections.
2.  Visualize Potential Energy Surfaces (PES) and understand their topology (valleys, saddle points).
3.  Simulate reaction trajectories to understand reaction probability.
4.  Explain the difference between "direct" and "complex-mode" reactions.

In [ ]:
# ============================================================
# GOOGLE COLAB SETUP
# ============================================================
import sys
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("=" * 60)
    print("RUNNING IN GOOGLE COLAB")
    print("=" * 60)

    # Clone repository to access images
    repo_url = "https://github.com/mcbadlon31/Reaction-Dynamics-Physical-Chemistry.git"

    print(f"\nCloning repository: {repo_url}")
    print("This may take a minute...")

    !git clone {repo_url} --depth 1 --quiet

    # Change to repository directory
    os.chdir('Reaction-Dynamics-Physical-Chemistry')

    # Install additional packages if needed
    print("\nInstalling additional packages...")
    !pip install -q seaborn plotly ipywidgets

    print("\n" + "=" * 60)
    print("[SUCCESS] Colab setup complete!")
    print("=" * 60)
    print(f"Current directory: {os.getcwd()}")
    print("\nYou can now run all cells normally.")
    print("Images will load from the cloned repository.")

else:
    print("=" * 60)
    print("RUNNING IN LOCAL JUPYTER ENVIRONMENT")
    print("=" * 60)
    print("\nNo setup needed - using local files")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import ipywidgets as widgets
from IPython.display import display

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 120,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'grid.alpha': 0.3
})

print("Libraries loaded.")

## 2. Molecular Beams and Velocity Selection

In a bulk gas, molecules have a Maxwell-Boltzmann distribution of speeds. To study energy dependence, we need a **monoenergetic** beam.

### Velocity Selector
A set of rotating disks with slots acts as a filter. Only molecules with a specific speed $v$ can pass through all slots without hitting a disk.
$$ v = \frac{L \omega}{\phi} $$
where:
-   $L$ is the distance between disks.
-   $\omega$ is the angular velocity (rad/s).
-   $\phi$ is the offset angle between slots.

In [ ]:
def velocity_selector(omega_rpm, L_m, phi_deg):
    omega = omega_rpm * 2 * np.pi / 60  # rad/s
    phi = np.radians(phi_deg)
    
    v_selected = (L_m * omega) / phi
    
    print(f"Angular Velocity: {omega_rpm} rpm")
    print(f"Length: {L_m} m")
    print(f"Offset Angle: {phi_deg} degrees")
    print(f"Selected Velocity: {v_selected:.2f} m/s")
    
    # Maxwell-Boltzmann distribution for N2 at 300 K
    v = np.linspace(0, 2000, 200)
    T = 300
    M = 0.028 # kg/mol
    R = constants.gas_constant
    f_v = (M/(2*np.pi*R*T))**1.5 * 4*np.pi * v**2 * np.exp(-M*v**2/(2*R*T))
    
    plt.figure(figsize=(8, 5))
    plt.plot(v, f_v, 'k-', label='Maxwell-Boltzmann (300 K)')
    plt.axvline(v_selected, color='r', linestyle='--', label='Selected Velocity')
    plt.xlabel('Velocity (m/s)')
    plt.ylabel('Probability Density')
    plt.title('Velocity Selection')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(velocity_selector, 
                 omega_rpm=widgets.FloatSlider(min=1000, max=20000, step=1000, value=5000, description='RPM'),
                 L_m=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, description='Length (m)'),
                 phi_deg=widgets.FloatSlider(min=1, max=90, step=1, value=10, description='Angle (deg)'));

### Angular Distribution (Differential Cross-Section)
When beams collide, products scatter in various directions. The **Differential Cross-Section** $I(\theta)$ tells us the probability of scattering into a specific angle $\theta$.
-   **Forward Scattering**: "Stripping" mechanism (direct reaction).
-   **Backward Scattering**: "Rebound" mechanism (direct reaction).
-   **Symmetric Scattering**: Formation of a long-lived complex.

In [ ]:
class MolecularBeamSimulator:
    """Simulate angular distribution of reaction products"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.mechanism_dropdown = widgets.Dropdown(
            options=['Stripping (Forward)', 'Rebound (Backward)', 'Complex (Symmetric)'],
            value='Stripping (Forward)',
            description='Mechanism:'
        )
        
    def plot(self, mechanism):
        theta = np.linspace(0, 2*np.pi, 360)
        
        # Define intensity distribution based on mechanism
        if 'Stripping' in mechanism:
            # Peak at 0 (Forward)
            I = np.exp(-5 * (theta)**2) + np.exp(-5 * (theta - 2*np.pi)**2)
            title = "Stripping Mechanism (Forward Scattering)"
            desc = "Reactant 'strips' an atom as it flies by. Direct reaction."
        elif 'Rebound' in mechanism:
            # Peak at pi (Backward)
            I = np.exp(-5 * (theta - np.pi)**2)
            title = "Rebound Mechanism (Backward Scattering)"
            desc = "Reactant hits head-on and bounces back. Direct reaction."
        else:
            # Symmetric (Forward + Backward)
            I = 0.5 * (np.exp(-2 * (theta)**2) + np.exp(-2 * (theta - np.pi)**2) + np.exp(-2 * (theta - 2*np.pi)**2))
            title = "Long-Lived Complex (Symmetric Scattering)"
            desc = "Molecules stick together, rotate, then break apart randomly."
            
        # Polar plot
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111, projection='polar')
        
        # Plot intensity
        ax.plot(theta, I, 'b-', linewidth=2)
        ax.fill(theta, I, alpha=0.3, color='blue')
        
        # Setup polar grid
        ax.set_theta_zero_location("E") # 0 degrees at right (Forward)
        ax.set_xticklabels(['Forward (0°)', '45°', '90°', '135°', 'Backward (180°)', '225°', '270°', '315°'])
        
        plt.title(title, pad=20)
        plt.show()
        
        print(f"\n💡 Interpretation: {desc}")
        
    def setup_display(self):
        widgets.interact(self.plot, mechanism=self.mechanism_dropdown)

print("\n🎯 Molecular Beam Simulator:")
beam_sim = MolecularBeamSimulator()

## 3. Potential Energy Surfaces (PES)

For a reaction $A + BC \rightarrow AB + C$, the potential energy depends on the three internuclear distances ($R_{AB}, R_{BC}, R_{AC}$). If we fix the angle (e.g., collinear), we can plot energy vs. $R_{AB}$ and $R_{BC}$.

### Features of a PES
1.  **Reactant Valley**: Region where $R_{AB}$ is large and $R_{BC}$ is near equilibrium bond length.
2.  **Product Valley**: Region where $R_{BC}$ is large and $R_{AB}$ is near equilibrium.
3.  **Transition State (Saddle Point)**: The highest energy point on the lowest energy path from reactants to products.

![PES Topology](images/pes_topology.png)

We use a simplified **LEPS (London-Eyring-Polanyi-Sato)** potential to model this surface.

In [ ]:
def leps_potential(r_ab, r_bc):
    # Simplified parameters for H + H2 -> H2 + H
    De = 4.75  # eV
    beta = 1.94  # A^-1
    r0 = 0.74  # A
    
    # Sato parameters (overlap)
    k = 0.15
    
    # Morse potential parts
    def Q(r):
        E = De * (np.exp(-2*beta*(r-r0)) - 2*np.exp(-beta*(r-r0)))
        return (E + (1-k)*E)/(1+k) # Simplified formulation for visualization

    def J(r):
        return 0.5 * De * (np.exp(-2*beta*(r-r0)) - 6 * np.exp(-beta*(r-r0)))
        
    r_ac = r_ab + r_bc # Collinear approximation
    
    Q_ab = Q(r_ab)
    Q_bc = Q(r_bc)
    Q_ac = Q(r_ac)
    J_ab = J(r_ab)
    J_bc = J(r_bc)
    J_ac = J(r_ac)
    
    term1 = Q_ab + Q_bc + Q_ac
    term2 = np.sqrt(((J_ab - J_bc)**2 + (J_bc - J_ac)**2 + (J_ac - J_ab)**2) / 2)
    
    return term1 - term2

def plot_pes(view_angle):
    x = np.linspace(0.5, 4.0, 50)
    y = np.linspace(0.5, 4.0, 50)
    X, Y = np.meshgrid(x, y)
    Z = leps_potential(X, Y)
    
    # Cutoff high energy for better visualization
    Z[Z > 5] = 5
    
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.9)
    ax.view_init(elev=30, azim=view_angle)
    ax.set_xlabel('R_AB (A)')
    ax.set_ylabel('R_BC (A)')
    ax.set_zlabel('Energy (eV)')
    ax.set_title('LEPS Potential Energy Surface')
    plt.show()

widgets.interact(plot_pes, 
                 view_angle=widgets.IntSlider(min=0, max=360, step=10, value=45, description='Rotate'));

### ❓ Concept Check

<details>
<summary><strong>Q: What does the saddle point on a PES represent?</strong></summary>

It represents the **Transition State**. It is a maximum in the direction of the reaction coordinate (the path from reactants to products) but a minimum in all other directions (perpendicular to the path).
</details>

## 4. Polanyi's Rules and Barrier Location

The location of the barrier (saddle point) on the PES has a profound effect on the reaction dynamics.

### Attractive vs. Repulsive Surfaces
-   **Early Barrier (Attractive)**: The barrier is located in the **reactant valley** (before the turn). 
    -   **Polanyi's Rule**: Translational energy is most effective at crossing an early barrier.
-   **Late Barrier (Repulsive)**: The barrier is located in the **product valley** (after the turn).
    -   **Polanyi's Rule**: Vibrational energy is most effective at crossing a late barrier.

Visualize the difference below:

In [ ]:
def heuristic_pes(r_ab, r_bc, barrier_type='early'):
    # Simple harmonic wells for visualization
    V_react = (r_bc - 1.0)**2
    V_prod = (r_ab - 1.0)**2
    
    # Base potential (valley)
    # Smooth min: -log(exp(-V1) + exp(-V2))
    V_base = -np.log(np.exp(-2*V_react) + np.exp(-2*V_prod)) / 2
    
    # Barrier Location
    if barrier_type == 'early':
        # Barrier in entrance channel
        r_ab_ts = 2.5
        r_bc_ts = 1.0
        amp = 0.5
    elif barrier_type == 'late':
        # Barrier in exit channel
        r_ab_ts = 1.0
        r_bc_ts = 2.5
        amp = 0.5
    else: # Symmetric
        r_ab_ts = 1.8
        r_bc_ts = 1.8
        amp = 0.8
        
    V_barrier = amp * np.exp(-1.5*((r_ab - r_ab_ts)**2 + (r_bc - r_bc_ts)**2))
    
    # Repulsive wall at small distances
    V_repulse = 0.5 * np.exp(-2*(r_ab + r_bc - 1.5))
    
    return V_base + V_barrier + V_repulse

def plot_pes_type(barrier_type):
    x = np.linspace(0.5, 4.0, 50)
    y = np.linspace(0.5, 4.0, 50)
    X, Y = np.meshgrid(x, y)
    Z = heuristic_pes(X, Y, barrier_type)
    
    Z[Z > 3] = 3 # Cutoff
    
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.9)
    ax.view_init(elev=40, azim=45)
    ax.set_xlabel('R_AB (A)')
    ax.set_ylabel('R_BC (A)')
    ax.set_title(f'PES: {barrier_type.capitalize()} Barrier')
    plt.show()

widgets.interact(plot_pes_type, 
                 barrier_type=widgets.Dropdown(options=['early', 'symmetric', 'late'], value='symmetric', description='Barrier Type'));

## 5. Reaction Trajectories

A reaction can be viewed as the motion of a point mass on this PES. The equations of motion (Newton's laws) determine the path.

### Factors Influencing Reaction
1.  **Kinetic Energy**: Does the system have enough energy to climb over the saddle point?
2.  **Vibrational Energy**: Is the energy in the right mode? (Polanyi's Rules)
    -   **Early Barrier**: Translational energy is more effective.
    -   **Late Barrier**: Vibrational energy is more effective.

### Simulation
The plot below shows a simplified 2D trajectory. 
-   **Red**: Reactive trajectory (crosses barrier).
-   **Blue**: Non-reactive trajectory (reflects back).

In [ ]:
def plot_trajectory(E_kin):
    x = np.linspace(0.5, 4.0, 100)
    y = np.linspace(0.5, 4.0, 100)
    X, Y = np.meshgrid(x, y)
    Z = leps_potential(X, Y)
    Z[Z > 5] = 5
    
    plt.figure(figsize=(8, 8))
    # Contour plot of PES
    plt.contour(X, Y, Z, levels=25, cmap='viridis')
    plt.xlabel('R_AB (A)')
    plt.ylabel('R_BC (A)')
    plt.title(f'Reaction Trajectory (E_kin = {E_kin} eV)')
    
    # Simulated trajectory (heuristic for demonstration)
    # Start at large R_AB (Reactants A + BC)
    path_x = np.linspace(4.0, 1.2, 50)
    path_y = np.linspace(0.74, 0.74, 50) # BC at equilibrium
    
    barrier_height = 0.4 # Approx barrier in this model
    
    if E_kin > barrier_height:
        # Reactive: Turn corner to products
        plt.plot(path_x, path_y, 'r-', linewidth=2)
        
        # Corner turn
        corner_x = np.linspace(1.2, 0.8, 20)
        corner_y = np.linspace(0.74, 1.2, 20)
        plt.plot(corner_x, corner_y, 'r-', linewidth=2)
        
        # Product valley exit
        exit_x = np.linspace(0.8, 0.8, 30)
        exit_y = np.linspace(1.2, 4.0, 30)
        plt.plot(exit_x, exit_y, 'r-', linewidth=2, label='Reactive')
        
    else:
        # Non-reactive: Reflect
        plt.plot(path_x, path_y, 'b-', linewidth=2)
        plt.plot(path_x[::-1], path_y, 'b--', linewidth=2, label='Non-reactive (Reflected)')
        
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(plot_trajectory, 
                 E_kin=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.3, description='Kinetic Energy'));

## 6. State-to-State Dynamics

We don't just care *if* a reaction happens, but *how* energy is distributed in the products. 
$$ A + BC(v, j) \rightarrow AB(v', j') + C $$

-   **Repulsive Surface (Late Barrier)**: Energy release tends to go into product **translation**.
-   **Attractive Surface (Early Barrier)**: Energy release tends to go into product **vibration** ($v'$).

This is the principle behind **Chemical Lasers** (e.g., HF laser), which rely on producing molecules in highly excited vibrational states (population inversion).

In [ ]:
class StateDistributionExplorer:
    """Explore product energy distribution"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.surface_type = widgets.Dropdown(
            options=['Attractive (Early Barrier)', 'Repulsive (Late Barrier)'],
            value='Attractive (Early Barrier)',
            description='PES Type:'
        )
        
    def plot(self, surface_type):
        # Vibrational states v'
        v_states = np.arange(0, 6)
        
        if 'Attractive' in surface_type:
            # Population inversion (high v')
            population = np.array([0.05, 0.1, 0.2, 0.4, 0.2, 0.05])
            title = "Attractive Surface -> Vibrational Excitation"
            desc = "Early barrier releases energy as the bond forms, 'whipping' the atoms into vibration. Good for lasers!"
        else:
            # Thermal-like (low v')
            population = np.array([0.6, 0.25, 0.1, 0.04, 0.01, 0.0])
            title = "Repulsive Surface -> Translational Excitation"
            desc = "Late barrier releases energy as the products fly apart, leading to high kinetic energy."
            
        plt.figure(figsize=(8, 5))
        bars = plt.bar(v_states, population, color='purple', alpha=0.7)
        
        plt.xlabel("Product Vibrational State (v')")
        plt.ylabel("Relative Population")
        plt.title(title)
        plt.xticks(v_states)
        plt.ylim(0, 0.7)
        plt.grid(axis='y', alpha=0.3)
        
        # Add labels
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.2f}', ha='center', va='bottom')
            
        plt.show()
        print(f"\n💡 Insight: {desc}")
        
    def setup_display(self):
        widgets.interact(self.plot, surface_type=self.surface_type)

print("\n📈 State-to-State Dynamics Explorer:")
explorer = StateDistributionExplorer()

## Summary

1.  **Molecular Beams**: Allow us to study single collision events and select reactant velocities.
2.  **PES**: The energy landscape determines the reaction mechanism. The saddle point is the transition state.
3.  **Dynamics**: Reaction probability depends on how energy is distributed (translation vs. vibration) and the topology of the surface (Polanyi's Rules).
4.  **State-to-State**: Energy release on attractive surfaces leads to vibrational excitation; repulsive surfaces lead to translation.